In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter

import dsds.transform as t
import dsds.fs as fs

from sklearn.preprocessing import power_transform
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 100, n_informative = 60, n_redundant = 40)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-6.961292,-1.354354,-0.869323,-0.884865,5.121076,0.724037,7.336044,4.882869,-2.07427,7.31135,-1.341896,-1.005311,-3.350232,12.06997,-7.901832,-2.49579,0.722828,0.81133,8.002032,3.065417,-0.643361,0.17594,24.239391,-0.286594,1.723879,-1.670237,1.858778,-7.094129,5.976315,-8.272599,-14.875249,-4.569537,0.656853,1.971732,8.967511,2.029471,…,-8.850719,-0.459343,4.358428,-0.118502,2.755337,-1.9843,2.940229,-1.795241,5.494203,0.162606,7.492647,-32.460988,-0.716955,-2.083133,3.100216,12.844987,18.51868,6.837905,-11.028648,-1.896649,-2.084956,1.739713,1.764852,4.881562,5.619424,-2.563158,-8.602555,-11.079209,0.467076,3.097124,0.376822,2.517541,5.388037,-4.397354,-30.778848,-5.041965,-6.983435
0,20.632676,4.249556,-4.094372,-4.175684,2.170555,5.160355,-3.605906,11.846278,-0.472618,0.456665,2.517982,4.272224,-1.400248,17.086646,4.171059,-4.407619,0.310268,2.526467,-4.822629,5.412142,-3.983946,9.655712,-0.55033,0.720188,-1.786544,-14.436858,-3.144479,2.803706,-6.448455,-2.132567,46.874545,-3.608811,3.07202,-8.625982,5.132065,-0.346496,…,20.466732,-1.638719,-2.821152,2.597876,-6.913894,2.524518,15.869481,9.230095,22.172403,-4.718264,-11.17447,7.256363,2.586593,-10.483316,3.160471,3.211078,-1.746299,3.871675,-10.271884,-0.374647,8.436675,2.664681,-8.877909,-27.951752,-4.736805,6.800557,-28.660288,-30.310261,7.586333,-1.962256,21.580478,5.230979,1.473096,-4.431784,-3.254034,1.852028,-0.416061
0,0.760132,-4.347395,13.496234,-0.431562,2.138826,-5.571485,-8.481878,-17.562283,4.484584,0.926601,2.038731,-5.724222,8.243527,18.518476,-2.995804,-4.136955,4.245631,2.139901,7.773933,6.064747,-6.169005,1.091385,-16.920923,2.593477,-1.66853,13.341464,-1.038949,-4.342841,-0.539217,3.028989,-21.56884,0.617118,-2.592063,-2.183553,-1.545164,2.025243,…,16.880737,3.006935,3.264033,-0.17796,3.374395,4.586409,39.581319,0.582877,4.443706,-2.106167,-16.744476,-13.865333,-0.157971,-1.702791,-0.548618,-14.920604,15.676817,-4.339007,0.898013,6.473017,-11.858197,0.083215,16.842015,-23.145912,4.977613,1.380346,-1.854526,-19.415052,-2.12596,-2.555243,-13.063444,2.753809,5.378086,-3.71934,-14.239367,5.956584,1.086044
0,-15.607154,8.736904,-13.413893,-2.351322,0.91337,-3.232315,-0.149811,-4.529269,12.054057,11.146986,3.01012,2.995094,-0.89265,22.822297,0.035335,-7.34237,-0.737487,-3.250542,10.457545,-7.300357,-4.349097,5.08922,8.35002,-2.533354,3.86165,35.771758,2.672292,-6.899604,1.852812,-7.04754,-17.233301,1.23183,-4.376885,-0.357091,5.777176,1.215443,…,3.272181,1.180432,-4.457339,-0.617199,5.325624,-1.060831,-27.461943,-4.689634,43.039389,-2.997954,-19.442372,-11.82962,0.73711,0.3284,-3.131322,11.988391,-4.207496,-6.484493,-11.792607,1.890237,-16.094931,1.70052,31.931763,10.673168,-6.90798,-10.729616,-42.774313,6.895033,-0.036809,-1.488856,-28.929027,0.208436,-0.51967,-0.629804,28.637312,-1.935074,0.6843
0,51.113307,-1.698436,5.345558,-5.836382,-6.737174,-0.864245,7.334694,-12.092255,-6.189048,-1.692949,-1.790517,-10.122019,10.569299,-5.552243,-7.174021,-0.

## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Fscore
2. Mutual Information Score
3. MRMR feature selection strategies
4. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Fscore

In [4]:
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",120.482724,5.1459e-28
"""column_1""",0.148875,0.699613
"""column_2""",13.568901,0.00023
"""column_3""",4762.418313,0.0
"""column_4""",1205.527117,1.4193e-262
"""column_5""",0.330126,0.565586
"""column_6""",0.18548,0.666707
"""column_7""",215.540639,9.5491e-49
"""column_8""",1501.147984,0.0


In [5]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.14s in computing Fscore.


,feature,f_value,p_value
0,column_0,120.482724,5.145915e-28
1,column_1,0.148875,6.996134e-01
2,column_2,13.568901,2.300851e-04
3,column_3,4762.418313,0.000000e+00
4,column_4,1205.527117,1.419270e-262
5,column_5,0.330126,5.655862e-01
6,column_6,0.185480,6.667065e-01
7,column_7,215.540639,9.549144e-49
8,column_8,1501.147984,0.000000e+00
9,column_9,0.612286,4.339301e-01


In [6]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.06s in computing Fscore.


,feature,f_value,p_value
0,column_0,120.482724,5.145915e-28
1,column_1,0.148875,6.996134e-01
2,column_2,13.568901,2.300851e-04
3,column_3,4762.418313,0.000000e+00
4,column_4,1205.527117,1.419270e-262
5,column_5,0.330126,5.655862e-01
6,column_6,0.185480,6.667065e-01
7,column_7,215.540639,9.549144e-49
8,column_8,1501.147984,0.000000e+00
9,column_9,0.612286,4.339301e-01


# Mutual Information Score

In [7]:
# Took 16s
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 100/100 [00:07<00:00, 13.07it/s]


feature,estimated_mi
str,f64
"""column_71""",0.054561
"""column_45""",0.038992
"""column_61""",0.030293
"""column_70""",0.027282
"""column_3""",0.025794
"""column_27""",0.024912
"""column_16""",0.024655
"""column_17""",0.024287
"""column_77""",0.022761


In [8]:
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [9]:
# Took 1m38s seconds by using sklearn. The reason is that sklearn's implementation did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality.
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_71""",0.054561
"""column_45""",0.038992
"""column_61""",0.030293
"""column_70""",0.027282
"""column_3""",0.025794
"""column_27""",0.024912
"""column_16""",0.024653
"""column_17""",0.024283
"""column_77""",0.022761


# MRMR Feature selection Strategy

In [10]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [11]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [12]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 50/50 [00:11<00:00,  4.31it/s]

Spent 14.99s to compute mrmr.


['column_71',
 'column_45',
 'column_61',
 'column_73',
 'column_35',
 'column_17',
 'column_27',
 'column_70',
 'column_16',
 'column_3',
 'column_77',
 'column_4',
 'column_48',
 'column_22',
 'column_24',
 'column_65',
 'column_11',
 'column_41',
 'column_94',
 'column_50',
 'column_8',
 'column_26',
 'column_49',
 'column_40',
 'column_66',
 'column_87',
 'column_76',
 'column_14',
 'column_75',
 'column_34',
 'column_54',
 'column_84',
 'column_72',
 'column_82',
 'column_28',
 'column_68',
 'column_21',
 'column_32',
 'column_67',
 'column_19',
 'column_12',
 'column_10',
 'column_55',
 'column_58',
 'column_38',
 'column_18',
 'column_81',
 'column_60',
 'column_85',
 'column_43']

In [13]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:00<00:00, 199.37it/s]

Output is sorted in order of selection (relevance).
Spent 0.30s in computing.


['column_71',
 'column_45',
 'column_61',
 'column_73',
 'column_35',
 'column_17',
 'column_27',
 'column_70',
 'column_16',
 'column_3',
 'column_77',
 'column_4',
 'column_48',
 'column_22',
 'column_24',
 'column_65',
 'column_11',
 'column_41',
 'column_94',
 'column_50',
 'column_8',
 'column_26',
 'column_49',
 'column_40',
 'column_66',
 'column_87',
 'column_76',
 'column_14',
 'column_75',
 'column_34',
 'column_54',
 'column_84',
 'column_72',
 'column_82',
 'column_28',
 'column_68',
 'column_21',
 'column_32',
 'column_67',
 'column_19',
 'column_12',
 'column_10',
 'column_55',
 'column_58',
 'column_38',
 'column_18',
 'column_81',
 'column_60',
 'column_85',
 'column_43']

In [14]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:01<00:00, 44.55it/s]

Output is sorted in order of selection (relevance).
Spent 1.14s in computing.


['column_71',
 'column_45',
 'column_61',
 'column_73',
 'column_35',
 'column_17',
 'column_27',
 'column_70',
 'column_16',
 'column_3',
 'column_77',
 'column_4',
 'column_48',
 'column_22',
 'column_24',
 'column_65',
 'column_11',
 'column_41',
 'column_94',
 'column_50',
 'column_8',
 'column_26',
 'column_49',
 'column_40',
 'column_66',
 'column_87',
 'column_76',
 'column_14',
 'column_75',
 'column_34',
 'column_54',
 'column_84',
 'column_72',
 'column_82',
 'column_28',
 'column_68',
 'column_21',
 'column_32',
 'column_67',
 'column_19',
 'column_12',
 'column_10',
 'column_55',
 'column_58',
 'column_38',
 'column_18',
 'column_81',
 'column_60',
 'column_85',
 'column_43']

# Power Transform

In [15]:
# Eager transform.
start = perf_counter()
res_eager = t.power_transform(df, target = target, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Inferring best paramters: 100%|██████████| 100/100 [00:01<00:00, 62.99it/s]


Spent 1.71s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,-6.910359,-1.351891,-0.869098,-0.88454,5.135647,0.723615,7.357852,4.835974,-2.074317,7.32964,-1.334915,-1.003968,-3.35045,11.73864,-7.92085,-2.467892,0.724629,0.810821,7.932222,3.060099,-0.643134,0.17591,23.820978,-0.286578,1.733228,-1.674336,1.864495,-7.143451,5.98582,-8.282941,-15.006336,-4.573764,0.655663,1.972301,9.010237,2.016192,…,-8.822073,-0.459457,4.369498,-0.118528,2.777947,-2.004875,2.947238,-1.801467,5.467058,0.162566,7.516031,-32.067935,-0.719408,-2.069728,3.095476,12.78998,18.268663,6.768092,-11.062769,-1.895492,-2.096334,1.737557,1.769492,4.905455,5.562916,-2.576577,-8.561843,-10.996363,0.466908,3.104547,0.37709,2.520685,5.417346,-4.436607,-30.506747,-5.027774,-6.944788
0,20.879844,4.266197,-4.091221,-4.171141,2.174185,5.14852,-3.598696,11.669488,-0.472622,0.456827,2.538602,4.287746,-1.400299,16.544806,4.16396,-4.337747,0.310635,2.522779,-4.855132,5.399028,-3.978666,9.624051,-0.551286,0.720281,-1.776671,-14.556925,-3.130809,2.792352,-6.437798,-2.133774,46.257082,-3.611716,3.054535,-8.620062,5.150394,-0.347039,…,20.562119,-1.639852,-2.815672,2.590166,-6.818545,2.494114,15.960135,9.146623,21.966272,-4.735319,-11.132459,7.303895,2.563607,-10.314787,3.155579,3.204426,-1.752875,3.842789,-10.302647,-0.374586,8.333455,2.660261,-8.817327,-27.65463,-4.780633,6.738083,-28.437554,-29.973153,7.567243,-1.958791,21.787447,5.241157,1.476567,-4.471524,-3.244015,1.854805,-0.415752
0,0.761391,-4.330224,13.51521,-0.431476,2.142367,-5.584848,-8.454881,-17.877371,4.48442,0.927199,2.053195,-5.699839,8.242646,17.912436,-2.999979,-4.073725,4.283131,2.137108,7.707072,6.049124,-6.158566,1.090464,-17.179901,2.594353,-1.659749,13.235143,-1.036888,-4.36595,-0.539061,3.026831,-21.789318,0.616983,-2.605376,-2.182878,-1.542624,2.01201,…,16.953656,3.003798,3.271003,-0.178017,3.405885,4.506896,39.89731,0.582048,4.424204,-2.110756,-16.669683,-13.743713,-0.158109,-1.693281,-0.548846,-14.989045,15.47881,-4.373939,0.897439,6.481384,-12.030726,0.083208,16.995756,-22.91606,4.930771,1.375698,-1.850984,-19.231549,-2.128488,-2.549847,-12.962162,2.757457,5.407312,-3.749409,-14.145495,5.975007,1.087854
0,-15.441895,8.787097,-13.395112,-2.349538,0.91417,-3.23796,-0.149789,-4.571421,12.053333,11.181101,3.037805,3.003861,-0.892673,22.014259,0.035334,-7.189477,-0.735625,-3.25613,10.353996,-7.321123,-4.343029,5.077184,8.257987,-2.532512,3.897299,35.358466,2.682814,-6.946895,1.854239,-7.055672,-17.39464,1.231357,-4.407969,-0.357065,5.799189,1.20997,…,3.278337,1.179794,-4.445905,-0.617815,5.390547,-1.067724,-27.269859,-4.719732,42.536831,-3.006205,-19.350241,-11.732743,0.734542,0.328873,-3.136149,11.938578,-4.235589,-6.549665,-11.830205,1.891388,-16.361907,1.698447,32.301978,10.750952,-6.986393,-10.854023,-42.394479,6.936381,-0.036811,-1.4867,-28.623719,0.208469,-0.519144,-0.631187,28.886574,-1.932087,0.685083
0,51.949831,-1.694788,5.350348,-5.828724,-6.715053,-0.864828,7.356496,-12.277498,-6.189317,-1.69122,-1.779017,-10.064732,10.568028,-5.659208,-7.19

In [16]:
res_lazy = t.power_transform(df.lazy(), target = target, strategy="yeo_johnson")

Inferring best paramters: 100%|██████████| 100/100 [00:01<00:00, 64.87it/s]


In [17]:
# Sklearn with Pandas

# In this case, the benefits of using dsds are:
# 1. Much better performance.
# 2. Users need to do extra steps to put data back in dataframe format.

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 7.40s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,0,-6.910359,-1.351891,-0.869098,-0.884540,5.135647,0.723615,7.357852,4.835974,-2.074317,...,-10.996363,0.466908,3.104547,0.377090,2.520685,5.417346,-4.436607,-30.506747,-5.027774,-6.944788
1,0,20.879844,4.266197,-4.091221,-4.171141,2.174185,5.148520,-3.598696,11.669488,-0.472622,...,-29.973153,7.567243,-1.958791,21.787447,5.241157,1.476567,-4.471524,-3.244015,1.854805,-0.415752
2,0,0.761391,-4.330224,13.515210,-0.431476,2.142367,-5.584848,-8.454881,-17.877371,4.484420,...,-19.231549,-2.128488,-2.549847,-12.962162,2.757457,5.407312,-3.749409,-14.145495,5.975007,1.087854
3,0,-15.441895,8.787097,-13.395112,-2.349538,0.914170,-3.237960,-0.149789,-4.571421,12.053333,...,6.936381,-0.036811,-1.486700,-28.623719,0.208469,-0.519144,-0.631187,28.886574,-1.932087,0.685083
4,0,51.949831,-1.694788,5.350348,-5.828724,-6.715053,-0.864828,7.356496,-12.277498,-6.189317,...,-2.497890,5.623152,-0.251799,11.500579,-3.339222,-2.029049,3.029241,13.132047,-8.736423,-2.059018


# GC

In [18]:
import gc 
gc.collect()

899